In [ ]:
import numpy as np
import pandas as pd
from pdfminer.high_level import extract_text
import re
import spacy
from spacy import displacy
from bs4 import BeautifulSoup
import json

In [ ]:
filename = "../data/exhibit_01.pdf"
text = extract_text(filename)
content2 = text
splits = content2.split('\n\n')

In [ ]:
# might be different for each pdf
splits = splits[1:]
speak_content = []
all_speakers = []

i = 0
while i < len(splits):
    part = splits[i]
    if part == 'HALDEMAN:' or part == 'PRESIDENT:':
        all_speakers.append(part[:-1])
        i += 1
        continue
    if part == '' or part == '*'*10:
        i += 1
        continue
    if part[:len('JUNE 23, 1972')] == 'JUNE 23, 1972':
        i += 2
        continue
    if len(part) != len(part.replace('\x0c', '')):
        part = part.replace('\x0c', '')
        if part[:len('JUNE 23, 1972')] == 'JUNE 23, 1972':
            i += 2
            continue
        else:
            speak_content[-1] = speak_content[-1] + '\n' + part
            i += 1
            continue
    speak_content.append(part)
    i += 1

In [ ]:
speak_content = speak_content[2:]

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def get_entities(sent):
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append([ent.text, ent.label_, ent.start_char, ent.end_char])
    return entities

def tagging(content):
    soup = BeautifulSoup('<p></p>','html.parser')
    entities = get_entities(content)
    if len(entities) == 0:
        soup.p.append(content)
    else:
        for i, entity in enumerate(entities):
            if i == 0:
                soup.p.append(content[0:entity[2]])
            else:
                soup.p.append(content[entities[i-1][3]:entity[2]])
            new_tag = soup.new_tag('span')  # Create a new tag
            new_tag['class'] = 'entity-'+entity[1]
            new_tag.append(content[entity[2]:entity[3]])
            soup.p.append(new_tag)
        soup.p.append(content[entities[-1][3]:])
    return soup

In [ ]:
conversion_json = {'lines':[]}
for speaker, paragraph in zip(all_speakers, speak_content):
    soup = tagging(paragraph)
    conversion_json['lines'].append({'speaker': speaker, 'words': str(soup)})

In [ ]:
json_object = json.dumps(conversion_json, indent=4)
 
# Writing to sample.json
with open("../data_json/exhibit_01.json", "w") as outfile:
    outfile.write(json_object)